In [1]:
import numpy as np
import pandas as pd
import spacy
import inflect
import re

ModuleNotFoundError: No module named 'spacy'

In [ ]:
df2 = pd.read_csv('english sentences 1.csv')

KeyError: ignored

In [ ]:
df2 = df2.drop_duplicates(subset='Sentence', keep='first')
#df2['Words list'] = [x.split() for x in df2['Sentence']]

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
df4 = df2

In [ ]:
# Initialize inflect engine
p = inflect.engine()

# A helper function to convert ordinals to words
def ordinal_to_words(match):
    number = int(match.group(0)[:-2])  # Convert the ordinal number to an integer
    return p.number_to_words(p.ordinal(number))  # Convert to ordinal word

# A helper function to convert currency to words
def currency_to_words(match):
    amount = match.group(0)[1:]  # Extract the amount without the currency symbol
    currency_word = 'dollars' if match.group(0).startswith('$') else 'euros'  # You can extend this logic to other currencies
    return p.number_to_words(amount) + ' ' + currency_word

# Define a function to preprocess the sentence
def preprocess_sentence(sentence):
    # Convert currency to words
    sentence = re.sub(r'\$\d+', currency_to_words, sentence)

    # Convert ordinal numbers to words
    sentence = re.sub(r'\b\d+st\b|\b\d+nd\b|\b\d+rd\b|\b\d+th\b', ordinal_to_words, sentence)

    # Convert cardinal numbers to words
    sentence = ' '.join([p.number_to_words(token) if token.isdigit() else token for token in sentence.split()])

    # Parse the sentence using the loaded model
    doc = nlp(sentence)

    # Initialize an empty list to store the tokens
    tokens = []

    # Iterate over each token
    for token in doc:
        # Keep tokens that are words (no punctuation, spaces, etc.)
        if token.is_alpha:
            tokens.append(token.text.lower())

    # Return the cleaned sentence as a string
    return ' '.join(tokens)

# Apply the preprocessing function to the 'Sentence' column of the DataFrame
df4['Cleaned_Sentence'] = df4['Sentence'][:5000].apply(preprocess_sentence)
